# AllegroGraph Ollama Integration

In this tutorial we assume you have `ollama` installed. If not, see the README about more information or go [here](https://ollama.com/download?utm_id=tutorials_awb-accessing-llms-jupyter-notebook-ollama) to download `ollama`. We will be using their `llama3.1:latest` model for this demo. Pull it using the following shell command:

```shell
ollama pull llama3.1:latest
```

We start by connecting to an AllegroGraph repository

In [1]:
from franz.openrdf.connect import ag_connect
from franz.openrdf.query.query import QueryLanguage

#Please set your connection parameters here
conn = ag_connect('ollama-example', create=True, clear=True, user='', password='', host='', port='')

Now we will set our connection parameters:

In [2]:
scheme = 'http'
ollama_host = 'localhost'
ollama_port = '11434'
model = 'llama3.1:latest'

We insert all parameters into a `SPARQL` prefixes list that will be included in each subsequent query.

In [3]:
PREFIXES = f"""
PREFIX franzOption_llmVendor: <franz:ollama>  
PREFIX franzOption_llmScheme: <franz:{scheme}>  
PREFIX franzOption_llmHost: <franz:{ollama_host}>  
PREFIX franzOption_llmPort: <franz:{ollama_port}>  
PREFIX franzOption_llmChatModel: <franz:{model}>  
 
PREFIX llm: <http://franz.com/ns/allegrograph/8.0.0/llm/>   
"""

## llm:response

We will start with our first magic predicate, `llm:response` which returns a list of answers

In [5]:
query_string = f"""
    {PREFIXES}

    SELECT ?book WHERE {{
        ?book llm:response "list the books in the Lord of the Rings trilogy" }}"""
df = conn.executeTupleQuery(query_string).toPandas()
df

,book
0,The Fellowship of the Ring
1,The Two Towers
2,The Return of the King


*note:* remember that the prefixes are a part of the SPARQL query string.

In [5]:
print(query_string)


    
PREFIX franzOption_llmVendor: <franz:ollama>  
PREFIX franzOption_llmScheme: <franz:http>  
PREFIX franzOption_llmHost: <franz:localhost>  
PREFIX franzOption_llmPort: <franz:11434>  
PREFIX franzOption_llmChatModel: <franz:llama3.1:latest>  
 
PREFIX llm: <http://franz.com/ns/allegrograph/8.0.0/llm/>   


    SELECT ?book WHERE {
        ?book llm:response "list the books in the Lord of the Rings trilogy" }


Here is a variety of other examples your can try. Please uncomment the desired clause and check the results! 

In [6]:
query_string = f"""
    {PREFIXES}

    SELECT ?response WHERE {{
        #?book llm:response "list the books in the Lord of the Rings trilogy"
        ?response llm:response "Who are the original 9 of the fellowship of the ring?"
        #?response llm:response "who eventually destroys the ring of power in the lord of the rings?"
        #?response llm:response "ADD YOUR OWN QUESTION HERE"
        }}"""
df = conn.executeTupleQuery(query_string).toPandas()
df

,response
0,Frodo
1,Sam
2,Aragorn
3,Legolas
4,Gimli
5,Boromir
6,Meriadoc Brandybuck
7,Peregrin Took
8,Gandalf


## Resources

When building datasets, we need to be able to turn string responses into IRIs which will be used as subjects in statements. In order to make an IRI out of a response string, use function **llm:node**. It converts a string into a unique fixed-size IRI in a deterministic way.

In [7]:
query_string = f"""
    {PREFIXES}

    SELECT ?book ?node WHERE {{
        ?book llm:response "What are the 3 books in the Lord of the Rings Trilogy. Please return only the name" .
        BIND ( llm:node(?book) as ?node ) . }}"""
df = conn.executeTupleQuery(query_string).toPandas()
df

,book,node
0,The Fellowship of the Ring,<http://franz.com/ns/allegrograph/8.0.0/llm/no...
1,The Two Towers,<http://franz.com/ns/allegrograph/8.0.0/llm/no...
2,The Return of the King,<http://franz.com/ns/allegrograph/8.0.0/llm/no...


## Creating a table of outputs

This query demonstrates the combination of both the response magic property and the response function to produce a table

In [8]:
query_string = f"""
    {PREFIXES}
    SELECT ?character ?race ?age ?location WHERE {{
        {{ SELECT ?character WHERE {{
            ?character llm:response "Who are the 9 original characters in the Fellowship of the Ring in the Lord of the Rings" . }} }}

        BIND(llm:response(CONCAT("What is the race of ", ?character, " in the Lord of the Rings. Return the race name only. No explanation needed")) as ?race) .
        BIND(llm:response(CONCAT("How old is ", ?character, " in the Lord of the Rings. Please return their age only as interger only. No explanation needed")) as ?age) .
        BIND(llm:response(CONCAT("Where is ", ?character, " from in Middle Earth in the Lord of the Rings. Please return only a place name. No explanation needed")) as ?location) .
    }}"""
df = conn.executeTupleQuery(query_string).toPandas()
df

,character,race,age,location
0,Frodo Baggins,Hobbit,53,"Hobbiton, The Shire"
1,Samwise Gamgee,Hobbit,87,Hobbiton
2,Aragorn,Dúnedain,87,Dúnedain
3,Legolas,Woodland Elf,139,Mirkwood
4,Gimli,Dwarves,139,Khazad-dûm
5,Boromir,Dunedain,35,Gondor
6,Meriadoc Brandybuck,Hobbit,130,Oxfordshire (Buckland)
7,Peregrin Took,Hobbit,17,Rivendell
8,Gandalf,Maia,1050,Valinor


# Graph Insertion

We will now insert the data from the previous query into the graph and show some sample SPARQL queries showing the data is present in the graph! Notice that we also have to create node for each character.

In [75]:
query_string = f"""
    {PREFIXES}
    PREFIX f: <http://franz.com/>

    INSERT {{
        ?characterNode a f:Character ;
                  rdfs:label ?character ;
                  f:race ?race ;
                  f:age ?age ;
                  f:placeOfOrigin ?location . }}
     WHERE {{
        {{ SELECT ?characterNode ?character WHERE {{
            ?character llm:response "Who are the 9 original characters in the Fellowship of the Ring in the Lord of the Rings" .
            BIND ( llm:node(?character) as ?characterNode ) . }}
        }}

        BIND(llm:response(CONCAT("What is the race of ", ?character, " in the Lord of the Rings. Return the race name only. No explanation needed")) as ?race) .
        BIND(llm:response(CONCAT("How old is ", ?character, " in the Lord of the Rings. Please return their age only as interger only. No explanation needed")) as ?age) .
        BIND(llm:response(CONCAT("Where is ", ?character, " from in Middle Earth in the Lord of the Rings. Please return only a place name. No explanation needed")) as ?location) .
    }}"""
conn.executeUpdate(query_string)
conn.size()

45

We will now query to find which characters are a "Hobbit"

In [76]:
query_string = """
    PREFIX f: <http://franz.com/>
    SELECT DISTINCT ?name WHERE {
        ?character f:race "Hobbit" ;
                   rdfs:label ?name . }"""
df = conn.executeTupleQuery(query_string).toPandas()
df

,name
0,Pippin Took
1,Merry Brandybuck
2,Samwise Gamgee
3,Frodo Baggins


If you want more examples of creating a graph using LLMs, please visit [our documentation](https://franz.com/agraph/support/documentation/llm-examples.html#Query-4---Borders)